In [1]:
!pip install faiss-cpu

In [2]:
!pip install sentence-transformers


  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached huggingface_hub-0.33.4-py3-none-any.whl.metadata (14 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
Using cached huggingface_hub-0.33.4-py3-none-any.whl (515 kB)
Using cached torch-2.7.1-cp312-cp312-win_amd64.whl (216.1 MB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   --------------------------

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


**Create source embeddings for the text column**

In [8]:
from sentence_transformers import SentenceTransformer


In [9]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
vectors.shape

(8, 768)

In [10]:
dim = vectors.shape[1]
dim

768

**Build a FAISS Index for vectors**

In [11]:
import faiss

index = faiss.IndexFlatL2(dim)

**Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index**

In [12]:
index.add(vectors)


In [13]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001AC3536CE40> >

**Encode search text using same encorder and normalize the output vector**

In [33]:
search_query = "looking for a place to visit during the holidays"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [34]:
svec

array([[ 1.77043136e-02,  7.70549998e-02, -1.25192413e-02,
        -1.39336148e-03,  5.92108816e-03, -1.42679159e-02,
        -4.72584032e-02,  3.91954929e-03,  4.41600867e-02,
         3.09807677e-02,  9.15983468e-02,  2.02603433e-02,
        -2.16004848e-02,  1.33208549e-02, -4.14275937e-02,
         4.16395590e-02,  4.37019207e-02,  6.06423570e-03,
         2.21780222e-02, -1.94715634e-02, -3.44190449e-02,
         3.79219875e-02,  5.27907768e-03,  2.22010948e-02,
        -1.11636976e-02, -2.53351294e-02,  7.79389217e-02,
         3.47347707e-02,  1.20056299e-02, -6.52653500e-02,
        -4.83941361e-02, -3.03445403e-02, -6.99562579e-02,
        -2.53461916e-02,  1.78119137e-06,  1.50222424e-02,
         2.15191673e-02,  2.21124496e-02, -8.38034526e-02,
         3.70956119e-03, -2.21117977e-02, -8.53062496e-02,
         6.44870917e-04,  1.62597820e-02, -1.88166052e-02,
         3.99218425e-02, -3.13138962e-02,  2.47449353e-02,
        -7.98756257e-03,  6.11144677e-02, -1.47846313e-0

In [35]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [36]:
distances, I = index.search(svec, k=2)

In [37]:
I

array([[6, 7]], dtype=int64)

In [38]:
df.loc[I[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [39]:
search_query

'looking for a place to visit during the holidays'

In [40]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel
